

In case you want to learn how ETL is done, please run the following notebook first and update the file name below accordingly

https://github.com/IBM/coursera/blob/master/coursera_ml/a2_w1_s3_ETL.ipynb


In [1]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200210000749-0000
KERNEL_ID = e2003eeb-1fd9-430a-9f93-05c0e3e3e13d
--2020-02-10 00:07:52--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet [following]
--2020-02-10 00:07:52--  https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving to: 'hmp.parquet'

100%[======================================>] 932,997     --.-K/s   in 0.04s   

2020-02-10 00:07:53 (22.1 MB/s

In [2]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [3]:
df_test.show()

+---+---+---+--------------------+--------------+
|  x|  y|  z|              source|         class|
+---+---+---+--------------------+--------------+
|  0| 11| 38|Accelerometer-201...| Sitdown_chair|
|  0| 27| 41|Accelerometer-201...|   Brush_teeth|
|  0| 29| 17|Accelerometer-201...|     Getup_bed|
|  0| 29| 25|Accelerometer-201...|  Climb_stairs|
|  0| 29| 37|Accelerometer-201...|   Brush_teeth|
|  0| 29| 38|Accelerometer-201...|   Brush_teeth|
|  0| 30| 24|Accelerometer-201...| Standup_chair|
|  0| 30| 36|Accelerometer-201...|  Climb_stairs|
|  0| 31| 32|Accelerometer-201...| Standup_chair|
|  0| 31| 35|Accelerometer-201...|   Brush_teeth|
|  0| 32| 23|Accelerometer-201...|     Getup_bed|
|  0| 32| 33|Accelerometer-201...|          Walk|
|  0| 32| 33|Accelerometer-201...|   Brush_teeth|
|  0| 32| 35|Accelerometer-201...|  Climb_stairs|
|  0| 32| 36|Accelerometer-201...|Descend_stairs|
|  0| 32| 36|Accelerometer-201...|  Climb_stairs|
|  0| 32| 42|Accelerometer-201...|   Brush_teeth|


In [4]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer


indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)


 



In [5]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [6]:

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer,lr])


In [7]:
model = pipeline.fit(df_train)

In [8]:
prediction = model.transform(df_train)

In [9]:
prediction.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- z: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- class: string (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- features_norm: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction) 

0.20602216483451916

In [11]:
prediction = model.transform(df_test)

In [12]:
binEval.evaluate(prediction) 

0.2089299340185825